# Clase Práctica 03

# Neural Network - Redes Neuronales Artificiales

# Clasificación Multiclase de Especies Florales

En este tutorial de proyecto, descubrirá cómo puede usar Keras para desarrollar y evaluar modelos de redes neuronales para problemas de clasificación de  múltiples clases.


# Base de datos Iris

En este tutorial utilizaremos el conjunto de datos de flores Iris. Este conjunto de datos está bien estudiado y es un buen problema para practicar las redes neuronales porque las 4 variables de entrada son numéricas y tienen la misma escala en centímetros. Cada instancia describe las propiedades de una observación.

Las mediciones de flujo y la variable de salida son especies específicas de flores. Los atributos para este conjunto de datos se pueden resumir de la siguiente manera:

1. Longitud del sépalo en centímetros.
2. Ancho sepal en centímetros.
3. Longitud del pétalo en centímetros.
4. Ancho del pétalo en centímetros.
5. Clase.

Este es un problema de clasificación multiclase, lo que significa que hay más de dos clases para predecir, de hecho, hay tres especies de flores. Este es un tipo de problema importante para practicar con redes neuronales porque los tres valores de clase requieren un manejo especializado (manejo de etiquetas). A continuación se observa una muestra de las primeros cinco atributos de las 150 instancias:

5.1,3.5,1.4,0.2,Iris-setosa

4.9,3.0,1.4,0.2,Iris-setosa

4.7,3.2,1.3,0.2,Iris-setosa

4.6,3.1,1.5,0.2,Iris-setosa

5.0,3.6,1.4,0.2,Iris-setosa


La clasificación de la base de datos de Iris es un problema bien estudiado y podemos esperar una precisión del modelo en el rango del 95% al 97%. Esto proporciona un buen objetivo para el desarrollo de nuestros modelos. 


# Importar Clases y Funciones

Vamos a comenzar importando todas las clases y funciones que necesitaremos para esta práctica. Esto incluye tanto la funcionalidad que requerimos de Keras, como la carga de datos desde Pandas, así como la preparación de datos y la evaluación del modelo desde Scikit-Learn.

Además, necesitamos inicializar el generador de números aleatorios (seed) a un valor constante. Esto es importante para garantizar que los resultados que vamos a obtener de este modelo se puedan repetir nuevamente de manera precisa. Esto asegura que el proceso estocástico del entrenamiento de un modelo de red neuronal pueda reproducirse.

In [ ]:
# librerías
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
# fix random seed for reproducibility
seed = 9
numpy.random.seed(seed)

# Cargar la base de datos Iris

El conjunto de datos se puede cargar directamente. Debido a que la variable de salida contiene cadenas (strings), es más fácil cargar los datos utilizando pandas. Luego podemos dividir los atributos (columnas) en variables de entrada (X) y variables de salida (Y).

In [ ]:
# base de datos 
dataframe = pandas.read_csv("iris.csv", header=None)
dataset = dataframe.values
X = dataset[:,0:4].astype(float)
Y = dataset[:,4]
print(X)
print("formato de datos de entrada")
print(X.shape)
print("formato de datos de salida")
print(Y.shape)
print(Y)

# Codificar las etiquetas de la salida

La variable de salida contiene tres valores diferentes (string). Al modelar problemas de clasificación multiclase usando redes neuronales, conviene remodelar el atributo de salida para que sea una matriz con un valor booleano para cada valor de clase. Esto se denomina codificación en caliente (**one hot encoding**) o creación de variables ficticias a partir de una variable categórica. Por ejemplo, en este problema, los tres valores de clase son Iris-setosa, Iris-versicolor e Iris-virginica. Si tuviéramos las tres observaciones:

Iris-setosa

Iris-versicolor

Iris-virginica

Podemos convertir las clases en una matriz binaria codificada para cada instancia de datos, esta se vería de la siguiente manera:

Iris-setosa, Iris-versicolor, Iris-virginica

1,                  0,              0

0,                  1,              0

0,                  0,              1

Podemos realizar esta conversión utilizando la clase LabelEncoder de scikit-learn. Luego se convierte el vector de enteros en una codificación one hot encoding utilizando la función Keras to_categorical().


In [ ]:
# codificar las clases a integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convertir clases (one hot encoded)
#print(encoded_Y)
dummy_y = np_utils.to_categorical(encoded_Y)
print("revise el formato de salida después de aplicar one hot encoded")
print(dummy_y)


# Definir el modelo de red neuronal

Keras proporciona clases para usar modelos de redes neuronales desarrollados con Keras y Scikit-Learn como vimos en la práctica 2. Hay una clase KerasClassifier en Keras que se puede usar como un Estimador en Scikit-Learn. KerasClassifier toma el nombre de una función como un argumento. Esta función debe devolver el modelo de red neuronal construido, listo para el entrenamiento.

A continuación se muestra una función que creará una red neuronal base para el problema de clasificación de la base Iris. Se crea una red simple completamente conectada que contiene 4 entradas con una capa oculta con el mismo número de neuronas (podría ser cualquier número de neuronas). La capa oculta utiliza una función de activación de rectificación (ReLU). Debido a que usamos una codificación one hot para la base de datos de Iris, la capa de salida debe crear 3 valores de salida, uno para cada clase. El valor de salida más grande se tomará como la clase predicha por el modelo. La topología de esta simple red neuronal se puede resumir como:

4 inputs -> [4 hidden nodes] -> 3 outputs

Se usa una función de activación sigmoide en la capa de salida, para asegurar que los valores de salida estén en el rango de 0 y 1, y se puedan usar como probabilidades pronosticadas. Finalmente, la red utiliza el algoritmo de optimización de descenso por gradiente de ADAM con una función de pérdida logarítmica, que se denomina **crossentropy categórica en Keras para clasificación multiclase**.

In [ ]:
# modelo multiclase
def multiclase_model():
    # modelo
    model = Sequential()
    model.add(Dense(4, input_dim=4, init='normal', activation='relu'))
    model.add(Dense(3, init='normal', activation='sigmoid'))
    # compilar
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) # categorical->multiclase
    return model

# Estimador

Ahora podemos crear nuestro KerasClassifier para su uso con Scikit-Learn. También podemos pasar argumentos en la construcción de la clase KerasClassifier que se pasará a la función fit() internamente utilizada para entrenar la red neuronal. Aquí, pasamos el número de épocas como 200 y tamaño de lote como 5 para usar al entrenar el modelo. La depuración también se desactiva cuando se entrena estableciendo verbose en 0.

In [ ]:
estimator = KerasClassifier(build_fn=multiclase_model, epochs=200, batch_size=5)

# Evaluar el modelo con la validación cruzada k-Fold

Ahora podemos evaluar el modelo de red neuronal en nuestros datos de entrenamiento. La biblioteca Scikit-Learn tiene una excelente capacidad para evaluar modelos utilizando un conjunto de técnicas. Una de las técnicas más usada para evaluar modelos de Deep Learning es la validación cruzada k-fold. Primero podemos definir el procedimiento de evaluación del modelo. Aquí, establecemos que el número de carpetas sea 10 y que mezclen los datos antes de particionarlos.

Luego podemos evaluar nuestro modelo (estimador) en nuestro conjunto de datos (X y dummy_y) utilizando el procedimiento de validación cruzada (**cross_val_score**) de 10 veces (kfold). La evaluación del modelo solo toma algunos minutos y devuelve un objeto que describe la evaluación de los 10 modelos construidos para cada una de las divisiones del conjunto de datos.

In [ ]:
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


# Código completo con k-fold


In [ ]:
# librerías
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
# seed
seed = 7
numpy.random.seed(seed)

# base de datos
dataframe = pandas.read_csv("iris.csv", header=None)
dataset = dataframe.values
X = dataset[:,0:4].astype(float)
Y = dataset[:,4]
#print(X)

# codificación
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# (one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
# print(dummy_y)

# definir modelos
def multiclase_model():
    # create model
    model = Sequential()
    model.add(Dense(4, input_dim=4, init='normal', activation='relu'))
    model.add(Dense(3, init='normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

estimator = KerasClassifier(build_fn=multiclase_model, epochs=200, batch_size=5)

kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

# Evaluar el modelo directamente sin k-fold

En este ejemplo se muestra como generar una red neuronal que clasifique multiples clases directamente, sin validación cruzada. Sin embargo, se utiliza un porcentaje de los datos de entrenamiento para entrenar y validar la red multiclase creada. 

In [ ]:
# librerías
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split
import matplotlib.pyplot as plt

# seed
seed = 7
numpy.random.seed(seed)

# base de datos
dataframe = pandas.read_csv("iris.csv", header=None)
dataset = dataframe.values
X = dataset[:,0:4].astype(float)
Y = dataset[:,4]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=seed) #20% para test

# codificación
encoder = LabelEncoder()
encoder.fit(y_train)
encoded_y_train = encoder.transform(y_train)
encoded_y_test = encoder.transform(y_test)
# (one hot encoded)
dummy_y_train = np_utils.to_categorical(encoded_y_train)
dummy_y_test = np_utils.to_categorical(encoded_y_test)

# modelo
model = Sequential()
model.add(Dense(4, input_dim=4, init='normal', activation='relu'))
model.add(Dense(3, init='normal', activation='softmax'))
# compilar
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# ajuste 
history = model.fit(X_train, dummy_y_train, validation_data=(X_test,dummy_y_test), nb_epoch=200, batch_size=5, verbose=1) 
# mostrar los datos que tiene historia
#print(history.history.keys())

# gráfico para la precisión obtenido de los datos de la historia 

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'ro', label='Acc Entrenamiento')
plt.plot(epochs, val_acc, 'b', label='Acc Validación')
plt.title('Accuracy: Entrenamiento and Validación')
plt.legend()
plt.savefig("Fig1.png")

# gráfico para la pérdida obtenido de los datos de la historia 
plt.figure()
plt.plot(epochs, loss, 'ro', label='Loss Entrenamiento')
plt.plot(epochs, val_loss, 'b', label='Loss Validación')
plt.title('Loss: Entrenamiento and Validación')
plt.legend()
plt.savefig("Fig2.png")
plt.show()

# evaluar
scores = model.evaluate(X_test,  dummy_y_test)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

# Revisión de predicción


A continuación revisaremos un pequeño ejemplo para realizar la predicción de un dato de entrada. 

Primero convertiremos la entrada al formato necesario para realizar la predicción usando numpy.reshape(). Recordar que el vector de entrada posee 4 entradas. 

La predicción realizada se compara con el valor de la etiqueta original. 

In [ ]:
indice = numpy.random.choice(list(range(len(X_test))), 1)[0]
prediccion = model.predict(numpy.reshape(X_test[indice], [1, -1])) # model.predict(numpy.reshape(X[0], (1, 4))) es lo mismo
etiqueta_real = dummy_y_test[indice]

print("Comparación de una predicción aleatoria")
print("Valor de la etiqueta real:")
print(etiqueta_real)
print("Valor de la predicción:")
print("se escoge el valor de predicción con mayor valor (probabilidad) para selecionar la clase")
print(prediccion)

#  Matriz de confusión 

La matriz de confusión se utiliza para describir el rendimiento de un modelo de clasificación en un conjunto de datos de prueba para los que se conocen los valores reales.

<img src="./images_tutoriales/confusion.png">


La matriz de confusión se compone de diferentes valores conocidos como: verdaderos positivos (TP), negativos verdaderos (TN), Falsos positivos (FP) y Falsos negativos (FN).

Positivos verdaderos (TP) : estos son los valores positivos pronosticados correctamente, lo que significa que el valor de la clase real es sí y el valor de la clase pronosticada también es sí. Por ejemplo, si el valor de clase real indica que un pasajero sobrevivió y la clase pronosticada le dice lo mismo.

Negativos verdaderos (TN) : estos son los valores negativos predichos correctamente, lo que significa que el valor de la clase real es no y el valor de la clase predicha también es no. Por ejemplo, si la clase real dice que este pasajero no sobrevivió y la clase pronosticada le dice lo mismo.

Falsos positivos y falsos negativos, estos valores se producen cuando su clase real se contradice con la clase predicha.

Falsos positivos (FP) : cuando la clase real es no y la clase predicha es sí. Por ejemplo, si la clase real dice que este pasajero no sobrevivió pero la clase pronosticada le dice que este pasajero sobrevivirá.

Falsos negativos (FN) : cuando la clase real es sí pero la clase predicha es no. Por ejemplo, si el valor real de la clase indica que este pasajero sobrevivió y la clase pronosticada le dice que el pasajero morirá.

Con estos cuatro parámetros, podemos calcular el accuracy, la precisión, la recuperación o sensibilidad (recall) y la puntuación de F1 (f1 score).

Accuracy (Exactitud): es la medida de rendimiento más intuitiva y es simplemente una proporción de observación pronosticada correctamente con respecto al total de observaciones. Uno puede pensar que, si tenemos una alta accuracy, nuestro modelo es el mejor. El accuracy es una gran medida, pero solo cuando tiene conjuntos de datos simétricos donde los valores de falso positivo y falso negativo son casi iguales. Por lo tanto, debe observar otros parámetros para evaluar el rendimiento de su modelo. 

Accuracy = TP + TN / (TP + FP + FN + TN)

Precisión : precisión es la relación entre las observaciones positivas pronosticadas correctamente y el total de observaciones positivas pronosticadas. La pregunta que responde esta métrica de todos los pasajeros que etiquetaron como sobrevivieron, ¿cuántos sobrevivieron realmente? ¿Qué proporción de identificaciones positivas fue correcta? Alta precisión se relaciona con la baja tasa de falsos positivos. 

Precisión = TP / (TP + FP)

Recall (Recuperación): Recall es la relación entre las observaciones positivas pronosticadas correctamente y todas las observaciones en la clase actual si (yes). La respuesta a la pregunta que se responde es: De todos los pasajeros que realmente sobrevivieron, ¿cuántos etiquetamos correctamente? ¿Qué proporción de positivos reales se identificó correctamente?

Recall = TP / (TP + FN)

F1 Score (Puntaje F1): El puntaje F1 es el promedio ponderado de Precisión y Recuperación. Por lo tanto, esta puntuación tiene en cuenta tanto los falsos positivos como los falsos negativos. El valor F1 se utiliza para combinar las medidas de precision y recall en un sólo valor. Esto es práctico porque hace más fácil el poder comparar el rendimiento combinado de la precisión y la exhaustividad entre varias soluciones. Intuitivamente, no es tan fácil de entender como la exactitud, pero la F1 suele ser más útil, especialmente si tiene una distribución de clases desigual. La exactitud funciona mejor si los falsos positivos y los falsos negativos tienen un costo similar. Si el costo de los falsos positivos y los falsos negativos es muy diferente, es mejor tener en cuenta tanto la precisión o el recall. 

Puntuación F1 = 2 * (Recall * Precision) / (Recall + Precision)

Por lo tanto, cada vez que construya un modelo, debe analizar estos parámetros para ver que tan bien se ha desempeñado su modelo.

Se implementará la matriz de confusión por medio de Sklearn. 

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

#Confution Matrix and Classification Report
Y_pred = model.predict(X_test)
y_pred = numpy.argmax(Y_pred, axis=1)

print('Confusion Matrix')
print(confusion_matrix(encoded_y_test, y_pred))
print('')
print('Classification Report')
target_names = list(['Iris-setosa','Iris-versicolor', 'Iris-verginica'])
print('')
print(classification_report(encoded_y_test, y_pred, target_names=target_names))